In [45]:
from pymongo import MongoClient
import pprint
import matplotlib.pyplot as plt

# import json
# import numpy as np

plt.style.use('ggplot')
%matplotlib inline

In [46]:
client = MongoClient()

In [47]:
db = client['bars_and_crime']
crime = db['crime']
bars = db['bars']
bar_data = db['bar_data']

In [69]:
db.bars.find_one()

{'_id': ObjectId('5e160f4b8555fe9b6b13e086'),
 'id': 'f5P9ufNH5mJz9BWj-sl8pw',
 'alias': 'the-beer-plant-austin',
 'name': 'The Beer Plant',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/LJVFPvjFq_XgwumfSIl93Q/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/the-beer-plant-austin?adjust_creative=0tMDijLbkodplsyFCw9huQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0tMDijLbkodplsyFCw9huQ',
 'review_count': 415,
 'categories': [{'alias': 'vegan', 'title': 'Vegan'},
  {'alias': 'beerbar', 'title': 'Beer Bar'},
  {'alias': 'gastropubs', 'title': 'Gastropubs'}],
 'rating': 4.0,
 'coordinates': {'latitude': 30.2966658, 'longitude': -97.7676046},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3110 Windsor Rd',
  'address2': '',
  'address3': None,
  'city': 'Austin',
  'zip_code': '78703',
  'country': 'US',
  'state': 'TX',
  'display_address': ['3110 Windsor Rd', 'Austin, TX 78703']},
 'phone': '+15125241800',
 'displa

In [77]:
# db.bars.count_documents({})

# len(bars.distinct('coordinates'))
# bars.distinct('id')

# bars.distinct('alias')
# 797 bars in ATX


result = bars.aggregate( 
                [{"$group": 
                      { "_id": {"id": "$id", "alias": "$alias", "name": "$name",
                                "review_count": "$review_count", "rating": "$rating",
                                "coordinates": "$coordinates", "price": "$price",
                                "location": "$location"}}}]
        )
print(type(result))
# for item in result:
#     pprint.pprint(item)


# for item in bars:
#     bar_data.insert(item)

<class 'pymongo.command_cursor.CommandCursor'>


In [25]:
item = crime.find_one({})
pprint.pprint(item)

row_count = crime.count()

{'_id': ObjectId('5e14eacb12b49b6f35cd1dcd'),
 'address': '2201 MONTOPLOIS DR',
 'apd_district': '',
 'apd_sector': '',
 'category_description': '',
 'census_tract': '',
 'clearance_date': '10/31/2018',
 'clearance_status': 'N',
 'district': '',
 'fam_violence': 'N',
 'high_offense_code': 2400,
 'high_offense_description': 'DATING DISTURBANCE',
 'incident_no': 20183000009,
 'lat': '',
 'lng': '',
 'location': '',
 'location_type': 'PARKING LOTS / GARAGE',
 'occurred_date': '10/27/2018',
 'occurred_date_time': '10/27/2018 12:03:00 AM',
 'occurred_time': 3,
 'pra': '',
 'report_date': '10/27/2018',
 'report_date_time': '10/27/2018 12:28:00 AM',
 'report_time': 28,
 'ucr_category': '',
 'x_coord': '',
 'y_coord': '',
 'zip': ''}


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  after removing the cwd from sys.path.
